In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.optim import Adam, SGD
from tqdm import tqdm
import csv
import cv2 as cv
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import random
import torch.nn.functional as F
import time
from transformers import SamModel, SamConfig, SamProcessor

# Portions of this code were generated with the assistance of ChatGPT (OpenAI, 2025) and subsequently modified by the author.
# OpenAI. (2025). ChatGPT (May 2025 version) [Large language model]. https://chat.openai.com

In [ ]:
# Load the model configuration
model_config = SamConfig.from_pretrained("D:/Thesis/SAM/segment-anything/sam_models/sam-vit-base")
processor = SamProcessor.from_pretrained("D:/Thesis/SAM/segment-anything/sam_models/sam-vit-base")

sam = SamModel(config=model_config)
#Update the model by loading the weights from saved file.
sam.load_state_dict(torch.load("D:/Thesis/SAM/segment-anything/notebooks/output_sam_b_brg_vessel_g_L2_v2_50.pth"))

# set the device to cuda if available, otherwise use cpu
device = "cuda" if torch.cuda.is_available() else "cpu"
sam.to(device)

In [ ]:
def apply_mask(im, mk):

    _, binary_mask = cv.threshold(mk, 20, 255, cv.THRESH_BINARY)

    masked_image = cv.bitwise_and(im, im, mask=binary_mask)

    # Combine them
    return masked_image

In [ ]:
import os
from PIL import Image
import numpy as np
import cv2 as cv
#Apply a trained model on large image

testDirImage = "D:/Downloads/Code/restormer_real_b2_Stijn_very/restormer_fundus_full_real_b2_alligned_ep100/test_latest/images/"
#testDirImage = "D:/Downloads/Stijn_it2/Stijn_bad_it2/restormer_fundus_full_real_b2_alligned_ep100/test_latest/images/"
#testDirImage = "D:/Downloads/Stijn/Stijn/Stijn_very_it2/restormer_fundus_full_real_b2_alligned_ep100/test_latest/images/"
#testDirImage = "D:/Downloads/C/C/"
testDirMask = "D:/Thesis/datasets/kaggle_drive/test/mask/"

testDirCircle = "D:/Thesis/datasets/kaggle_drive/test/circle_mask/"

files1 = sorted([f for f in os.listdir(testDirImage) if os.path.isfile(os.path.join(testDirImage, f))])
files2 = sorted([f for f in os.listdir(testDirMask) if os.path.isfile(os.path.join(testDirMask, f))])
files3 = sorted([f for f in os.listdir(testDirCircle) if os.path.isfile(os.path.join(testDirCircle, f))])

test_images = []
test_color_images = []
test_masks = []

j=0
for i in range(len(os.listdir(testDirImage))):
    if files1[i].endswith('.png'):
        img = cv.imread(cv.samples.findFile(testDirImage + files1[i]))
        #msk = cv.imread(cv.samples.findFile(testDirCircle + files3[j]), cv.IMREAD_GRAYSCALE)
        #img = apply_mask(img, msk)
        #cv.imwrite(testDirImage + files1[i], img)
        im = cv.cvtColor(img,cv.COLOR_BGR2RGB)
        red, green, blue = cv.split(im)
        img = np.asarray(green)
        test_images += [cv.resize(img,(200,200))] 
        test_color_images += [cv.resize(im,(256,256))]
        j += 1
        

for i in range(len(os.listdir(testDirMask))):
    if files2[i].endswith('.png'):
        img = cv.imread(cv.samples.findFile(testDirMask + files2[i]))
        img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
        img = cv.resize(img, (256, 256))  # (width, height)
        img = img / 255
        test_masks += [img]
        test_masks += [img]
        test_masks += [img]

random = np.random.randint(0,len(test_images))

import matplotlib.pyplot as plt

print(len(test_images))
print(len(test_masks))

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].imshow(np.array(test_color_images[random]), cmap='gray')
axes[1].imshow(np.array(test_masks[random]), cmap='gray')
plt.show()
#patches = patchify(large_test_image, (256, 256), step=256)  #Step=256 for 256 patches means no overlap

In [ ]:
# Define the size of your array
array_size = 200

# Define the size of your grid
grid_size =15

# Generate the grid points
x = np.linspace(0, array_size-1, grid_size)
y = np.linspace(0, array_size-1, grid_size)

# Generate a grid of coordinates
xv, yv = np.meshgrid(x, y)

# Convert the numpy arrays to lists
xv_list = xv.tolist()
yv_list = yv.tolist()

# Combine the x and y coordinates into a list of list of lists
input_points = [[[int(x), int(y)] for x, y in zip(x_row, y_row)] for x_row, y_row in zip(xv_list, yv_list)]
print (input_points)

#We need to reshape our nxn grid to the expected shape of the input_points tensor
# (batch_size, point_batch_size, num_points_per_image, 2),
# where the last dimension of 2 represents the x and y coordinates of each point.
#batch_size: The number of images you're processing at once.
#point_batch_size: The number of point sets you have for each image.
#num_points_per_image: The number of points in each set.
input_points = torch.tensor(input_points).view(1, 1, grid_size*grid_size, 2)

In [ ]:
def compute_iou(pred_mask, gt_mask, threshold=0.5):
    # Binarize the predicted mask if it contains probabilities
    pred_bin = (pred_mask > threshold).astype(np.uint8)
    gt_bin = (gt_mask > 0.5).astype(np.uint8)

    # Compute intersection and union
    intersection = np.sum(pred_bin & gt_bin)  # intersection (pred & gt)
    union = np.sum(pred_bin | gt_bin)        # union (pred | gt)

    # Compute IoU (intersection / union), with a small epsilon to avoid division by zero
    iou = intersection / (union + 1e-6)
    return iou

def accuracy(pred_mask, gt_mask, active_thresh=0.01):
    """
    pred_mask: np.ndarray of shape [1, H, W] or [1, 1, H, W]
    gt_mask:   np.ndarray of same shape

    Returns: scalar float accuracy over relevant pixels
    """
    if pred_mask.ndim == 4:
        pred_mask = pred_mask[0, 0]
        gt_mask = gt_mask[0, 0]
    elif pred_mask.ndim == 3:
        pred_mask = pred_mask[0]
        gt_mask = gt_mask[0]

    gt_mask = gt_mask[pred_mask > active_thresh]
    pred_mask = pred_mask[pred_mask > active_thresh]

    l = 1-np.abs(gt_mask - pred_mask)

    total = len(l)
    if total == 0:
        return 0.0

    accuracy = np.sum(l) / total
    return accuracy

In [ ]:
def compute_f1_ap(preds, targets, threshold=0.5, eps=1e-7):
    preds_binary = (preds > threshold).astype(np.uint8)
    targets = (targets > 0.5).astype(np.uint8)

    tp = np.sum(preds_binary * targets)
    fp = np.sum(preds_binary * (1 - targets))
    fn = np.sum((1 - preds_binary) * targets)

    precision = tp / (tp + fp + eps)
    recall = tp / (tp + fn + eps)
    f1 = 2 * precision * recall / (precision + recall + eps)

    return [f1, precision, recall]


In [ ]:
import warnings
warnings.filterwarnings("ignore", message="The following named arguments are not valid for `SamImageProcessor.preprocess`.*")

sum_f1 = 0
sum_iou = 0
sum_precision = 0
sum_recall = 0
# sum_map = [[]for i in range(10)]
# map_values = [round(0.5+i*0.05, 2) for i in range(10)]

sam.eval()
min_f1 = 1
max_f1 = 0
pic_min = None
pic_max = None
mk_min = None
mk_max = None
mk_min_og = None
mk_max_og = None
im_og = None
im_og = None
for i in range(len(test_images)):
    single_patch = Image.fromarray(test_images[i])

    inputs = processor(single_patch.convert("RGB"), input_points=input_points, return_tensors="pt")

    # Move the input tensor to the GPU if it's not already there
    inputs = {k: v.to(device) for k, v in inputs.items()}

    
    with torch.no_grad():
        outputs = sam(**inputs, multimask_output=False)

    # apply sigmoid
    single_patch_prob = torch.sigmoid(outputs.pred_masks.squeeze(1))

    single_patch_prob = single_patch_prob.cpu().detach().numpy().squeeze()
    f1 = compute_f1_ap(single_patch_prob, test_masks[i])[0]
    sum_f1 += f1
    #print("F1: ", f1)

    iou = compute_iou(single_patch_prob, test_masks[i])
    quality_score = np.mean([f1, iou])
    #print("IoU: ", iou)

    acc = accuracy(single_patch_prob, test_masks[i])

    #print("Acc: ", acc)
    # sum += acc

    if(quality_score < min_f1):
        min_f1 = quality_score
        pic_min = test_color_images[i]
        mk_min = single_patch_prob
        mk_min_og = test_masks[i]
    if(quality_score > max_f1):
        max_f1 = quality_score
        pic_max = test_color_images[i]
        mk_max = single_patch_prob
        mk_max_og = test_masks[i]


print("total F1: ", sum_f1/len(test_images))
#print("total Acc: ", sum_acc/len(test_images))
# for i in range(len(map_values)):
#     ap = np.mean(sum_map[i])
#     sum_mAP += ap
#     print("total AP for threshold ", map_values[i], ": ", ap)
# print("mAP: ", sum_mAP/len(map_values))


In [ ]:
# Display the images with min and max F1 scores
plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.imshow(pic_min)
plt.title(f'Min Quality Score: {min_f1:.4f}')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(mk_min, cmap='gray')
plt.axis('off')
plt.subplot(1, 3, 3)
plt.imshow(mk_min_og, cmap='gray')
plt.axis('off')

plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.imshow(pic_max)
plt.title(f'Max Quality Score: {max_f1:.4f}')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(mk_max, cmap='gray')
plt.axis('off')
plt.subplot(1, 3, 3)
plt.imshow(mk_max_og, cmap='gray')
plt.axis('off')